In [3]:
import sqlalchemy
sqlalchemy.__version__

'1.3.20'

In [4]:

from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
       return "<User(name='%s', fullname='%s', nickname='%s')>" % (
                            self.name, self.fullname, self.nickname)

In [5]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [6]:
Base.metadata.create_all(engine)

2020-10-26 15:52:13,490 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-26 15:52:13,491 INFO sqlalchemy.engine.base.Engine ()
2020-10-26 15:52:13,492 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-26 15:52:13,493 INFO sqlalchemy.engine.base.Engine ()
2020-10-26 15:52:13,494 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-10-26 15:52:13,494 INFO sqlalchemy.engine.base.Engine ()
2020-10-26 15:52:13,495 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-10-26 15:52:13,496 INFO sqlalchemy.engine.base.Engine ()
2020-10-26 15:52:13,497 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2020-10-26 15:52:13,498 INFO sqlalchemy.engine.base.Engine ()
2020-10-26 15:52:13,499 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [8]:
ed_user.name

'ed'

In [9]:
ed_user.nickname

'edsnickname'

In [10]:
str(ed_user.id)

'None'

In [12]:
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [13]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [14]:
session = Session()
session.add(ed_user)

In [15]:
our_user = session.query(User).filter_by(name='ed').first() 
our_user

2020-10-26 15:56:42,759 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-26 15:56:42,761 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-26 15:56:42,762 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2020-10-26 15:56:42,763 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2020-10-26 15:56:42,764 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [16]:
ed_user is our_user

True

In [17]:
our_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [18]:
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [19]:
session.add_all([
     User(name='wendy', fullname='Wendy Williams', nickname='windy'),
     User(name='mary', fullname='Mary Contrary', nickname='mary'),
     User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [20]:
ed_user.nickname = 'eddie'

In [21]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [22]:
session.new 

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [23]:
session.commit()

2020-10-26 15:59:49,291 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2020-10-26 15:59:49,292 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2020-10-26 15:59:49,293 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-26 15:59:49,294 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2020-10-26 15:59:49,294 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-26 15:59:49,295 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2020-10-26 15:59:49,296 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-26 15:59:49,296 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2020-10-26 15:59:49,297 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
ed_user.id 

2020-10-26 16:00:18,627 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-26 16:00:18,629 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-10-26 16:00:18,629 INFO sqlalchemy.engine.base.Engine (1,)


1

In [32]:
session.query(User).all()

2020-10-26 16:02:05,767 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-10-26 16:02:05,768 INFO sqlalchemy.engine.base.Engine ()


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>,
 <User(name='wendy', fullname='Wendy Williams', nickname='windy')>,
 <User(name='mary', fullname='Mary Contrary', nickname='mary')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [33]:
ed_user.name = 'Edwardo'

In [34]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [35]:
session.query(User).all()

2020-10-26 16:03:17,904 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2020-10-26 16:03:17,905 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2020-10-26 16:03:17,906 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-26 16:03:17,906 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2020-10-26 16:03:17,907 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-10-26 16:03:17,908 INFO sqlalchemy.engine.base.Engine ()


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='wendy', fullname='Wendy Williams', nickname='windy')>,
 <User(name='mary', fullname='Mary Contrary', nickname='mary')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [36]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2020-10-26 16:03:47,835 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2020-10-26 16:03:47,836 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [37]:
session.rollback()

2020-10-26 16:04:13,596 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [38]:
ed_user.name

2020-10-26 16:04:22,047 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-26 16:04:22,048 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-10-26 16:04:22,049 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [39]:
fake_user in session

False

In [40]:
session.query(User).all()

2020-10-26 16:04:43,966 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-10-26 16:04:43,967 INFO sqlalchemy.engine.base.Engine ()


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>,
 <User(name='wendy', fullname='Wendy Williams', nickname='windy')>,
 <User(name='mary', fullname='Mary Contrary', nickname='mary')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [41]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname, instance.id)

2020-10-26 16:05:16,674 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2020-10-26 16:05:16,675 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones 1
wendy Wendy Williams 2
mary Mary Contrary 3
fred Fred Flintstone 4


In [42]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2020-10-26 16:06:48,943 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2020-10-26 16:06:48,944 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone
